In [21]:
from imutils import face_utils
import numpy as np
import pyautogui as pyag
import imutils
import dlib
import cv2

# Initialize counters
EYEBROW_LIFT_CTR = 0
FRAME_CTR = 0

# Threshold values
SHORT_BLINK_FRAMES = 8
LONG_BLINK_FRAMES = 15
EYEBROW_LIFT_FRAMES = 3
LONG_EYEBROW_LIFT_FRAMES = 10

EYEBROW_THRESH = 0.5
EYE_AR_THRESH = 0.19

SCROLL_INIT = [False, -1]

INPUT_MODE = False
SCROLL_MODE = False

# Returns EAR given eye landmarks
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])

    # Compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = np.linalg.norm(eye[0] - eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # Return the eye aspect ratio
    return ear

def calc_eyebrow_lift_ratio(leftEye, leftEyeBrow, rightEye, rightEyeBrow) -> int:
    LeftPt1 = np.linalg.norm(leftEyeBrow[2] - leftEye[1])
    LeftPt2 = np.linalg.norm(leftEyeBrow[3] - leftEye[2])
    LeftPt3 = np.linalg.norm(leftEyeBrow[0] - leftEyeBrow[4])
    left = (((LeftPt1 + LeftPt2) / 2)/LeftPt3)
    
    RightPt1 = np.linalg.norm(rightEyeBrow[1] - rightEye[1])
    RightPt2 = np.linalg.norm(rightEyeBrow[2] - rightEye[2])
    RightPt3 = np.linalg.norm(rightEyeBrow[0] - rightEyeBrow[4])
    right = (((RightPt1 + RightPt2) / 2)/RightPt3)
    
    if left < right*0.75:
        right = left
    elif right < left*0.75:
        left = right
    eyebrow_ratio = (left + right) / 2
    
    return eyebrow_ratio

# MOUTH_AR_THRESH = 0.6
# MOUTH_AR_CONSECUTIVE_FRAMES = 15

# EYE_AR_CONSECUTIVE_FRAMES = 15
# WINK_AR_DIFF_THRESH = 0.04
# WINK_AR_CLOSE_THRESH = 0.19
# WINK_CONSECUTIVE_FRAMES = 10
# MOUTH_COUNTER = 0
# EYE_COUNTER = 0
# WINK_COUNTER = 0
# EYE_CLICK = False
# LEFT_WINK = False
# RIGHT_WINK = False

ANCHOR_POINT = (0, 0)
WHITE_COLOR = (255, 255, 255)
YELLOW_COLOR = (0, 255, 255)
RED_COLOR = (0, 0, 255)
GREEN_COLOR = (0, 255, 0)
BLUE_COLOR = (255, 0, 0)
BLACK_COLOR = (0, 0, 0)


shape_predictor = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(lbStart, lbEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(rbStart, rbEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]
(nStart, nEnd) = face_utils.FACIAL_LANDMARKS_IDXS["nose"]
#(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

vid = cv2.VideoCapture(0)
resolution_w = 1366
resolution_h = 768
cam_w = 1080
cam_h = 720
unit_w = resolution_w / cam_w
unit_h = resolution_h / cam_h

while True:
    # Grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale
    # channels)
    _, frame = vid.read()
    frame = cv2.flip(frame, 1)
    frame = imutils.resize(frame, width=cam_w, height=cam_h)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    rects = detector(gray, 0)

    # Loop over the face detections
    if len(rects) > 0:
        rect = rects[0]
    else:
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        continue
    FRAME_CTR += 1
    #cv2.putText(frame, str(FRAME_CTR), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)

    # Determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy
    # array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    # Extract the left and right eye coordinates, then use the
    # coordinates to compute the eye aspect ratio for both eyes
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftBrow = shape[lbStart:lbEnd]
    rightBrow = shape[rbStart:rbEnd]
    nose = shape[nStart:nEnd]

    # Because I flipped the frame, left is right, right is left.
    temp = leftEye
    leftEye = rightEye
    rightEye = temp
    
    temp = leftBrow
    leftBrow = rightBrow
    rightBrow = temp

    # Average the mouth aspect ratio together for both eyes
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    ear = (leftEAR + rightEAR) / 2.0
#     diff_ear = np.abs(leftEAR - rightEAR)
    cv2.putText(frame, "EYE: " + str(ear), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
    
    if ear > EYE_AR_THRESH:
        eyebrow_lifted = calc_eyebrow_lift_ratio(leftEye, leftBrow, rightEye, rightBrow)
        cv2.putText(frame, "EYEBROW_RATIO: " + str(eyebrow_lifted), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
        if eyebrow_lifted > EYEBROW_THRESH:
            EYEBROW_LIFT_CTR += 1
            if EYEBROW_LIFT_CTR == EYEBROW_LIFT_FRAMES:
                SCROLL_INIT[0] = True
                SCROLL_INIT[1] = FRAME_CTR
            if EYEBROW_LIFT_CTR == LONG_EYEBROW_LIFT_FRAMES:
                INPUT_MODE = not INPUT_MODE
                SCROLL_INIT[0] = False
                SCROLL_INIT[1] = 0
                
            cv2.putText(frame, "EYEBROW CTR: " + str(EYEBROW_LIFT_CTR), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
        else:
            EYEBROW_LIFT_CTR = 0
            
    if SCROLL_INIT[0] and FRAME_CTR - SCROLL_INIT[1] == 8:
        if SCROLL_MODE:
            SCROLL_MODE = False
        elif INPUT_MODE:
            SCROLL_MODE = True
        SCROLL_INIT[0] = False
        SCROLL_INIT[1] = 0
    
    if INPUT_MODE:
        cv2.putText(frame, "INPUT MODE ON", (200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, GREEN_COLOR, 2)
    else:
        SCROLL_MODE = False
        cv2.putText(frame, "INPUT MODE OFF", (200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
        
    if SCROLL_MODE:
        cv2.putText(frame, "SCROLL MODE ON", (200, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, GREEN_COLOR, 2)
        
    

    nose_point = (nose[3, 0], nose[3, 1])

    # Compute the convex hull for the left and right eye, then
    # visualize each of the eyes
    leftBrowHull = cv2.convexHull(leftBrow)
    rightBrowHull = cv2.convexHull(rightBrow)
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    cv2.drawContours(frame, [leftBrowHull], -1, YELLOW_COLOR, 1)
    cv2.drawContours(frame, [rightBrowHull], -1, YELLOW_COLOR, 1)
    cv2.drawContours(frame, [leftEyeHull], -1, YELLOW_COLOR, 1)
    cv2.drawContours(frame, [rightEyeHull], -1, YELLOW_COLOR, 1)

#     for (x, y) in np.concatenate((leftBrow, rightBrow, leftEye, rightEye), axis=0):
#         cv2.circle(frame, (x, y), 2, GREEN_COLOR, -1)
        
#     # Check to see if the eye aspect ratio is below the blink
#     # threshold, and if so, increment the blink frame counter
#     if diff_ear > WINK_AR_DIFF_THRESH:

#         if leftEAR < rightEAR:
#             if leftEAR < EYE_AR_THRESH:
#                 WINK_COUNTER += 1

#                 if WINK_COUNTER > WINK_CONSECUTIVE_FRAMES:
#                     pag.click(button='left')

#                     WINK_COUNTER = 0

#         elif leftEAR > rightEAR:
#             if rightEAR < EYE_AR_THRESH:
#                 WINK_COUNTER += 1

#                 if WINK_COUNTER > WINK_CONSECUTIVE_FRAMES:
#                     pag.click(button='right')

#                     WINK_COUNTER = 0
#         else:
#             WINK_COUNTER = 0
#     else:
#         if ear <= EYE_AR_THRESH:
#             EYE_COUNTER += 1

#             if EYE_COUNTER > EYE_AR_CONSECUTIVE_FRAMES:
#                 SCROLL_MODE = not SCROLL_MODE
#                 # INPUT_MODE = not INPUT_MODE
#                 EYE_COUNTER = 0

#                 # nose point to draw a bounding box around it

#         else:
#             EYE_COUNTER = 0
#             WINK_COUNTER = 0

#     if mar > MOUTH_AR_THRESH:
#         MOUTH_COUNTER += 1

#         if MOUTH_COUNTER >= MOUTH_AR_CONSECUTIVE_FRAMES:
#             # if the alarm is not on, turn it on
#             INPUT_MODE = not INPUT_MODE
#             # SCROLL_MODE = not SCROLL_MODE
#             MOUTH_COUNTER = 0
#             ANCHOR_POINT = nose_point

#     else:
#         MOUTH_COUNTER = 0

#     if INPUT_MODE:
#         cv2.putText(frame, "READING INPUT!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
#         x, y = ANCHOR_POINT
#         nx, ny = nose_point
#         w, h = 60, 35
#         multiple = 1
#         cv2.rectangle(frame, (x - w, y - h), (x + w, y + h), GREEN_COLOR, 2)
#         cv2.line(frame, ANCHOR_POINT, nose_point, BLUE_COLOR, 2)

#         dir = direction(nose_point, ANCHOR_POINT, w, h)
#         cv2.putText(frame, dir.upper(), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
#         drag = 20
#         if dir == 'right':
#             pyag.moveRel(drag, 0)
#         elif dir == 'left':
#             pyag.moveRel(-drag, 0)
#         elif dir == 'up':
#             if SCROLL_MODE:
#                 pyag.scroll(40)
#             else:
#                 pyag.moveRel(0, -drag)
#         elif dir == 'down':
#             if SCROLL_MODE:
#                 pyag.scroll(-40)
#             else:
#                 pyag.moveRel(0, drag)

#     if SCROLL_MODE:
#         cv2.putText(frame, 'SCROLL MODE IS ON!', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)

    # cv2.putText(frame, "MAR: {:.2f}".format(mar), (500, 30),
    #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, YELLOW_COLOR, 2)
    # cv2.putText(frame, "Right EAR: {:.2f}".format(rightEAR), (460, 80),
    #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, YELLOW_COLOR, 2)
    # cv2.putText(frame, "Left EAR: {:.2f}".format(leftEAR), (460, 130),
    #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, YELLOW_COLOR, 2)
    # cv2.putText(frame, "Diff EAR: {:.2f}".format(np.abs(leftEAR - rightEAR)), (460, 80),
    #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If the `Esc` key was pressed, break from the loop
    if key == 27:
        break

# Do a bit of cleanup
cv2.destroyAllWindows()
vid.release()


